<div class="alert alert-danger">
    Read the Instructions.ipynb notebook!
</div>

# 1. Inference-by-Enumeration (10 points)

The Inference-by-Enumeration algorithm computes the answer to a probabilistic query $P(\mathbf{X} \mid \mathbf{E})$ exactly from the full joint distribution table (FJDT).

---
### 1.1. Implementation


<div class="alert alert-warning">
Implement the Inference-by-Enumeration algorithm. (2 points)
</div>

Implement the `inference_by_enumeration` function for a generic probabilistic query of the form $P(\mathbf{X} \mid \mathbf{E})$. Note that this version of the Inference-by-Enumeration algorithm computes the probabilistic query for all possible assignments to the evidence variables, not only for one specific assignment (cf. slide deck: Probabilistic Models - Part 2: Fundamental Concepts and Notation, p. 40). The function must return one object:
- The answer to the probabilistic query, which is a `np.ndarray` with the same number of dimensions and the same variable order as the FJDT, but not the same size: The dimensions of non-query and non-evidence variables ($\mathbf{Z}$) must be converted to singleton dimensions, i.e., dimensions of size one.

For example, if we have a full joint distribution table of three binary variables (shape $2\times2\times2$) and we ask for the distribution of the first variable given the second variable, the resulting conditional distribution table would be of shape $2\times2\times1$.

**Hint:** Remember to solve this without a `for` loop. Set the `keepdims` parameter of NumPy's <a href="https://numpy.org/doc/stable/reference/generated/numpy.sum.html">sum</a> method to `True` to not discard the reduced dimensions. Keeping these empty dimensions simplifies <a href="https://numpy.org/doc/stable/user/basics.broadcasting.html">broadcasting operations</a> to a no-brainer.

In [1]:
import numpy as np
from helpers import print_table

In [7]:
def inference_by_enumeration(FJDT: np.ndarray, query_variable_indices: tuple, evidence_variable_indices: tuple=tuple()) -> np.ndarray:
    '''
    Computes the answer to a probabilistic query exactly from the full joint distribution table.
    :param table: The full joint distribution table as a np.ndarray.
    :param query_variable_indices: A tuple containing the indices of the query variables in the FJDT.
    :param evidence_variable_indices: A tuple containing the indices of the evidence variables in the FJDT.
    :returns: The answer to the probabilistic query; a `np.ndarray`.
    ''' 
    assert type(FJDT) == np.ndarray, "FJDT must be a np.ndarray"
    assert type(query_variable_indices) == tuple, "query_variable_indices must be a tuple"
    assert type(evidence_variable_indices) == tuple, "evidence_variable_indices must be a tuple"
    
    # compute the set of non-query and non-evidence variables, Z
    query_variables = query_variable_indices + evidence_variable_indices
    Z = tuple(set(range(FJDT.ndim)).difference(query_variables))

    # YOUR CODE HERE
    #sum out the non-query, non-evidence variables
    p1 = np.sum(FJDT, axis=Z, keepdims=True)
    #sum out the query variables
    normConst = np.sum(p1, axis=query_variable_indices, keepdims=True)
    # normalize
    return p1 / normConst

In [8]:
# create a full joint distribution table for three binary variables
ABC = np.ones((2,2,2)) / 8
# name the variable indices so we can refer to them more easily
A, B, C = 0, 1, 2

# check type & shape of result
assert type(inference_by_enumeration(ABC, (B, C), ())) == np.ndarray
# compute P(A)
assert inference_by_enumeration(ABC, (A,), ()).shape == (2, 1, 1)
# compute P(BC)
assert inference_by_enumeration(ABC, (B, C), ()).shape == (1, 2, 2)
# compute P(BC|A)
assert inference_by_enumeration(ABC, (B, C), (A,)).shape == (2, 2, 2)
# compute P(B|AC)
assert inference_by_enumeration(ABC, (B,), (C,A,)).shape == (2, 2, 2)


---
### 1.2. Full Joint Distribution Table

<br>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Atlantic_blue_marlin.jpg" width="500" height="600">
<br>

Based on his experience, Santiago, an old Cuban fisherman, has learned that temperature and precipitation are the most prominent factors influencing marlin fishing. After decades of (more or less) successful years, he decides to retire and pass on his knowledge to a young apprentice. Since the apprentice received excellent grades in her probabilistic models class, he creates the following full joint distribution table $P(C, R, H)$ and hands it over to her:


<table style="border-collapse:collapse;border-spacing:0;width:500px"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center" rowspan="2">$P({C}, {R}, {H})$</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$\neg r$<br></th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$r$</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$\neg c$<br></td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.25</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.39</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.30</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.01<br></td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$c$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.03</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.01</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.05</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.05</td></tr></table>

In this table, $C$, $R$, and $H$ are the binary random variables encoding catch, rain, and hot, respectively. 
    
    
**Hint**: You can use `print_table` to print your probability distribution tables in a similar fashion.

In [4]:
help(print_table)

Help on function print_table in module helpers:

print_table(probability_table: numpy.ndarray, variable_names: str) -> None
    Prints a probability distribution table.
    
    Parameters
    ----------
    probability_table : np.ndarray
        The probability distribution table
    variable_names : str
        A string containing the variable names, e.g., 'CDE'.
    
    Returns
    -------
    None



<div class="alert alert-warning">
Create a NumPy array that contains the full joint distribution table $P(C, R, H)$ as defined above. <b>Important</b>: Encode $C$, $R$, and $H$ in the first, second, and third dimension of the NumPy array, respectively. Use index 0 for event *False* and index 1 for event *True*. (1 point)
</div>

In [5]:
CRH = None
# Check the result with print_table(CRH, 'CRH')

# remove the placeholder
# YOUR CODE HERE
CRH = np.array([[[0.25, 0.39], [0.3, 0.01]], [[0.03, 0.01], [0.005, 0.005]]])
C_, R_, H_ = 0, 1, 2
print_table(CRH, 'CRH')

,$r_0$,$r_0$,$r_1$,$r_1$
,$h_0$,$h_1$,$h_0$,$h_1$
$c_0$,0.250,0.390,0.300,0.010
$c_1$,0.030,0.010,0.005,0.005


In [6]:
assert CRH is not None, 'Store the result into the variable \'CRH\'!'
assert CRH.shape == (2,2,2), 'The full joint distribution table must have shape (2,2,2)'
assert CRH.sum() == 1, 'The probabilities of all atomic events must sum to one.'


---
### 1.3. Probabilistic Queries


Compute the following two probabilistic queries. For each query, there are three tasks:
1. Write down the probabilistic query and compute the answer from the full joint distribution. Show all intermediate steps, but keep your answer short! Use $\LaTeX$ and Markdown.
2. Give the shape of the solution (without singleton dimensions) and the number of non-redundant entries in the result table, storing your answer into the provided variables.
3. Check your answer with the `inference_by_enumeration` function and store the result into the provided variable. Select the result for the given evidence and remove singleton dimensions.

<div class="alert alert-warning">
Compute the probability distribution over catching a marlin. (3 points)
</div>

$$P(c) = \sum_{x, y}^{}P(c, R = x, H = y) \\
= P(c,-r,-h)+ P(c,-r,h) + P(c,r,-h) + P(c,r,h) \\
=0.03+0.01+0.005+0.005 = 0.05$$

$$P(\neg c) = \sum_{x, y}^{}P(\neg c, R = x, H = y) \\
= P(\neg c,-r,-h)+ P(\neg c,-r,h) + P(\neg c,r,-h) + P(\neg c,r,h) \\
=0.25+0.39+0.30+0.01 = 0.95$$

$$P(C) = [0.95, 0.05]$$

In [20]:
probability_table_shape = (2,) # e.g., (2,2,2) for the FJDT, (2,) for a vector, () for a scalar
number_non_redundant_elements = 1 # e.g., 2*2*2 - 1 for the FJDT 
C = None # Use inference_by_enumeration to compute the result. Select the result for the given evidence (if any) and discard singleton dimensions.

# remove the placeholder
# YOUR CODE HERE<
C = inference_by_enumeration(CRH, (C_,), ()).squeeze()
print_table(C, 'C')

$c_0$,0.950
$c_1$,0.050


In [21]:
assert type(probability_table_shape) is tuple, 'Shape of the result must be a tuple.'
assert type(number_non_redundant_elements) is int, 'Number of elements must be int.'
assert C is not None, 'Store the result into the variable \'C\'!'


<div class="alert alert-warning">
Compute the probability distribution over catching a marlin given the temperature being hot. (3 points)
</div>

$$P(c | h) = \frac{P(c \wedge h)}{P(h)} = \\ \frac{0.01 + 0.005}{0.39+0.01+0.01+0.005} =  \\ 0.036$$

$$P(\neg c | h) = \frac{P(\neg c \wedge h)}{P(h)} = \\ \frac{0.39+0.01}{0.39+0.01+0.01+0.005} = \\ 0.964$$

$$P(C | h) = {0.964, 0.036}$$

In [23]:
probability_table_shape = (2,1,2) # e.g., (2,2,2) for the FJDT, (2,) for a vector, () for a scalar
number_non_redundant_elements = 1 # Assuming that we do not factor in the elements of !h, i.e. counting them as redundant
C_h = None # Use inference_by_enumeration to compute the result. Select the result for the given evidence (if any) and discard singleton dimensions.

# remove the placeholder
# YOUR CODE HERE
C_h = inference_by_enumeration(CRH, (C_,), (H_,))
print(C_h)

[[[0.94017094 0.96385542]]

 [[0.05982906 0.03614458]]]


In [10]:
assert type(probability_table_shape) is tuple, 'Shape of the result must be a tuple.'
assert type(number_non_redundant_elements) is int, 'Number of elements must be int.'
assert C_h is not None, 'Store the result into the variable \'C_h\'!'


---
### 1.4. Independence

<div class="alert alert-warning">
Show that catching the marlin is not independent of the weather being hot, i.e., $C \not \perp H$. Check your result with NumPy. (1 point)
</div>



**Hint**: It is sufficient to print the joint distribution and the product of the marginal distributions (use `print_table`.)

In [10]:
CH = None # store the joint distribution here
C_times_H = None # store the product of the marginal distributions here

# remove the placeholder
# YOUR CODE HERE
CH = inference_by_enumeration(CRH, (C_,H_,), ()).squeeze()
C_times_H = (inference_by_enumeration(CRH, (C_,), ()) * inference_by_enumeration(CRH, (H_,), ())).squeeze() 

print((CH == C_times_H).all())

print('Product of Marginals:')
print_table(CH, 'CH')

print('Joint Probability:')
print_table(C_times_H, 'CH')

False
Product of Marginals:


,$h_0$,$h_1$
$c_0$,0.550,0.400
$c_1$,0.035,0.015


Joint Probability:


,$h_0$,$h_1$
$c_0$,0.556,0.394
$c_1$,0.029,0.021


In [12]:
assert type(CH) == np.ndarray, 'Results must be NumPy arrays.'
assert type(C_times_H) == np.ndarray, 'Results must be NumPy arrays.'

assert CH.shape == (2,2), 'Results must be 2x2 arrays.'
assert C_times_H.shape == (2,2), 'Results must be 2x2 arrays.'